# Chapter8 텍스트 분석 ~04 - 실리콘밸리 5차 정기세션 과제

## 스팸 텍스트 메시지 데이터 활용 링크: https://www.kaggle.com/team-ai/spam-text-message-classification/download
### 다운로드 후 적절하게 파일명을 변경하세요

### 1. 텍스트 분석 수행 프로세스 및 텍스트 정규화 작업을 정리하세요

1. 텍스트 사전 준비작업
2. 피처 벡터화/추출
3. ML 모델 수립 및 학습/예측/평가

<텍스트 정규화 작업>
* 클렌징
* 토큰화
* 필터링/스톱 워드 제거/철자 수정
* Stemming
* Lemmatization

### 2. 해당 데이터의 index가 901인 문장에 대해 문장별 단어 토큰화(교재 함수 사용)를 수행하고 스톱 워드를 제거하세요

In [1]:
from nltk import sent_tokenize
from nltk import word_tokenize
import pandas as pd
import numpy as np

In [2]:
text_spam=pd.read_csv('SPAM text message 20170820 - Data.csv')

In [3]:
text_sample=text_spam.iloc[901,1]

In [4]:
def tokenize_text(text):
    
    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)
    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

#여러 문장들에 대해 문장별 단어 토큰화 수행. 
word_tokens = tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

<class 'list'> 2
[['Probably', 'money', 'worries', '.'], ['Things', 'are', 'coming', 'due', 'and', 'i', 'have', 'several', 'outstanding', 'invoices', 'for', 'work', 'i', 'did', 'two', 'and', 'three', 'months', 'ago', '.']]


In [5]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
# 위 예제의 3개의 문장별로 얻은 word_tokens list 에 대해 stop word 제거 Loop
for sentence in word_tokens:
    filtered_words=[]
    # 개별 문장별로 tokenize된 sentence list에 대해 stop word 제거 Loop
    for word in sentence:
        #소문자로 모두 변환합니다. 
        word = word.lower()
        # tokenize 된 개별 word가 stop words 들의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
    
print(all_tokens)

[['probably', 'money', 'worries', '.'], ['things', 'coming', 'due', 'several', 'outstanding', 'invoices', 'work', 'two', 'three', 'months', 'ago', '.']]


### 3.  해당 문장의 결과 토큰화 값들 중 진행형에 대해 Stemmimg과 Lemmatization을 수행 후 결과를 비교하세요

In [6]:
from nltk.stem import LancasterStemmer
stemmer=LancasterStemmer()

stemmer.stem('coming')

'com'

In [7]:
from nltk.stem import WordNetLemmatizer

lemma=WordNetLemmatizer()
lemma.lemmatize('coming','v')

'come'

### 4. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

### 4-1. 위의 스팸 데이터로 train, test set을 나누세요 (힌트: train_test_split)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_spam['Message'], text_spam['Category'], test_size=0.3, random_state=42)

### 4-2. 두 개의 피처 벡터화를 수행 후 각각의 벡터화 shape을 구하세요

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train , y_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (3900, 7262)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

print('학습 데이터 Text의 TF-IDF Shape:',X_train_tfidf_vect.shape)

학습 데이터 Text의 TF-IDF Shape: (3900, 7262)


### 4-3. 각각에 로지스틱 회귀 모형을 적용해 예측해 보세요

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CountVectorized Logistic Regression 의 예측 정확도는 0.984


In [12]:
# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.970


## Women's E-Commerce Clothing Reviews 데이터 링크: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

### 5. 파이프라인 사용 
### 5-1. 새로운 데이터 셋을 다운로드 후 'Review text'로 'Rating'을 예측하도록 적절한 전처리를 수행 후 test, train set으로 분할하세요

In [13]:
e=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')

In [14]:
ed=e[['Review Text','Rating']].dropna()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(ed['Review Text'], ed['Rating'], test_size=0.3, random_state=42)

### 5-2. 사이킷런의 Pipeline방식을 적용해 TF-IDF기반으로 벡터화하고 로지스틱 회귀 모형으로 예측해보세요(파라미터 튜닝은 디폴트로 해도 되고 맘대로 해보세요)

In [22]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([
    ('thidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf',LogisticRegression(random_state=156))
])

pipeline.fit(X_train, y_train)
pred=pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regreesion의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

Pipeline을 통한 Logistic Regreesion의 예측 정확도는 0.633
